In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("bea")
         .getOrCreate())

In [4]:
spark

In [5]:
rdd = spark.sparkContext.textFile('/labs/laba01/ml-100k/u.data')

In [8]:
rdd.take (5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [9]:
data = rdd.map(lambda x: x.split("\t"))

In [10]:
data.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [58]:
hist_film = sorted(data.filter(lambda x: x[1] == '318').map(lambda x: (x[2], 1)).countByKey().items())

In [59]:
hist_film

[('1', 4), ('2', 6), ('3', 23), ('4', 79), ('5', 186)]

In [60]:
hist_all = sorted(data.map(lambda x: (x[2], 1)).countByKey().items())

In [62]:
hist_all

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [65]:
final = {"hist_film": [x[1] for x in hist_film], "hist_all": [x[1] for x in hist_all]}

In [66]:
final

{'hist_film': [4, 6, 23, 79, 186],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [67]:
import json


with open('lab01.json', 'w') as f:
    json.dump(final, f, indent=2)
    print("New json file is created")

New json file is created


In [68]:
sc.stop()